# 14-813/18-813 Course Project – Option 2

### Task-I: Build and populate necessary tables

Firstly, we contruct the spark section.

In [1]:
# if you installed Spark on windows, 
# you may need findspark and need to initialize it prior to being able to use pyspark
# Also, you may need to initialize SparkContext yourself.
# Uncomment the following lines if you are using Windows!
#import findspark
#findspark.init()
#findspark.find()

import pyspark
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession

# from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("GenericAppName") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/25 22:18:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In the following, we import the MQTT training and test dataset and add a new column to identify them.

In [2]:
from pyspark import SparkFiles
from pyspark.sql.functions import lit


col_names = [
    "tcp.flags",
    "tcp.time_delta",
    "tcp.len",
    "mqtt.conack.flags",
    "mqtt.conack.flags.reserved",
    "mqtt.conack.flags.sp",
    "mqtt.conack.val",
    "mqtt.conflag.cleansess",
    "mqtt.conflag.passwd",
    "mqtt.conflag.qos",
    "mqtt.conflag.reserved",
    "mqtt.conflag.retain",
    "mqtt.conflag.uname",
    "mqtt.conflag.willflag",
    "mqtt.conflags",
    "mqtt.dupflag",
    "mqtt.hdrflags",
    "mqtt.kalive",
    "mqtt.len",
    "mqtt.msg",
    "mqtt.msgid",
    "mqtt.msgtype",
    "mqtt.proto_len",
    "mqtt.protoname",
    "mqtt.qos",
    "mqtt.retain",
    "mqtt.sub.qos",
    "mqtt.suback.qos",
    "mqtt.ver",
    "mqtt.willmsg",
    "mqtt.willmsg_len",
    "mqtt.willtopic",
    "mqtt.willtopic_len",
    "target"
]

#print (col_names.tolist())

# train_path = "/Users/ultrali/Documents/Grad/14813/project/archive/Data/FINAL_CSV/train70_augmented.csv"
# test_path = "/Users/ultrali/Documents/Grad/14813/project/archive/Data/FINAL_CSV/test30_augmented.csv"
train_path = "/Users/ultrali/Documents/Grad/14813/project/archive/Data/FINAL_CSV/train70_reduced.csv"
test_path = "/Users/ultrali/Documents/Grad/14813/project/archive/Data/FINAL_CSV/test30_reduced.csv"

df_train = spark.read.csv(train_path, header=True, inferSchema= True).toDF(*col_names)
df_test = spark.read.csv(test_path, header=True, inferSchema= True).toDF(*col_names)

In [3]:
df_train = df_train.withColumn("dataset", lit(0))
df_test = df_test.withColumn("dataset", lit(1))

df_train.show(1, vertical=True)
df_test.show(1, vertical=True)

print(df_train.count())
print(df_test.count())

22/10/25 22:37:37 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
-RECORD 0--------------------------------
 tcp.flags                  | 0x00000018 
 tcp.time_delta             | 0.998867   
 tcp.len                    | 10         
 mqtt.conack.flags          | 0          
 mqtt.conack.flags.reserved | 0.0        
 mqtt.conack.flags.sp       | 0.0        
 mqtt.conack.val            | 0.0        
 mqtt.conflag.cleansess     | 0.0        
 mqtt.conflag.passwd        | 0.0        
 mqtt.conflag.qos           | 0.0        
 mqtt.conflag.reserved      | 0.0        
 mqtt.conflag.retain        | 0.0        
 mqtt.conflag.uname         | 0.0        
 mqtt.conflag.willflag      | 0.0        
 mqtt.conflags              | 0          
 mqtt.dupflag               | 0.0        
 mqtt.hdrflags              | 0x00000030 
 mqtt.kalive                | 0.0        
 mqtt.len             

In [6]:
df_train.printSchema()

root
 |-- tcp.flags: string (nullable = true)
 |-- tcp.time_delta: double (nullable = true)
 |-- tcp.len: integer (nullable = true)
 |-- mqtt.conack.flags: string (nullable = true)
 |-- mqtt.conack.flags.reserved: double (nullable = true)
 |-- mqtt.conack.flags.sp: double (nullable = true)
 |-- mqtt.conack.val: double (nullable = true)
 |-- mqtt.conflag.cleansess: double (nullable = true)
 |-- mqtt.conflag.passwd: double (nullable = true)
 |-- mqtt.conflag.qos: double (nullable = true)
 |-- mqtt.conflag.reserved: double (nullable = true)
 |-- mqtt.conflag.retain: double (nullable = true)
 |-- mqtt.conflag.uname: double (nullable = true)
 |-- mqtt.conflag.willflag: double (nullable = true)
 |-- mqtt.conflags: string (nullable = true)
 |-- mqtt.dupflag: double (nullable = true)
 |-- mqtt.hdrflags: string (nullable = true)
 |-- mqtt.kalive: double (nullable = true)
 |-- mqtt.len: double (nullable = true)
 |-- mqtt.msg: string (nullable = true)
 |-- mqtt.msgid: double (nullable = true)
 |-

Before injesting the date (which will actually create the mqtt table), we need to revise the metadata and ensure certain constraints (data type of columns), as it mentioned in the handout.

Here we rename the columns according to programming conventions.

In [12]:
new_cols = [
    '_'.join(item.split('.')) for item in col_names[:-1]
]

renamed_columns_df = df_train
for i in range(len(new_cols)):
    renamed_columns_df = renamed_columns_df.withColumnRenamed(col_names[i], new_cols[i])

renamed_columns_df.printSchema()

root
 |-- tcp_flags: string (nullable = true)
 |-- tcp_time_delta: double (nullable = true)
 |-- tcp_len: integer (nullable = true)
 |-- mqtt_conack_flags: string (nullable = true)
 |-- mqtt_conack_flags_reserved: double (nullable = true)
 |-- mqtt_conack_flags_sp: double (nullable = true)
 |-- mqtt_conack_val: double (nullable = true)
 |-- mqtt_conflag_cleansess: double (nullable = true)
 |-- mqtt_conflag_passwd: double (nullable = true)
 |-- mqtt_conflag_qos: double (nullable = true)
 |-- mqtt_conflag_reserved: double (nullable = true)
 |-- mqtt_conflag_retain: double (nullable = true)
 |-- mqtt_conflag_uname: double (nullable = true)
 |-- mqtt_conflag_willflag: double (nullable = true)
 |-- mqtt_conflags: string (nullable = true)
 |-- mqtt_dupflag: double (nullable = true)
 |-- mqtt_hdrflags: string (nullable = true)
 |-- mqtt_kalive: double (nullable = true)
 |-- mqtt_len: double (nullable = true)
 |-- mqtt_msg: string (nullable = true)
 |-- mqtt_msgid: double (nullable = true)
 |-

Then we determine the data type of each columns, depending on the data it contains and the meaning it carries.

However, we discovered that the data is quite clean in terms of the data type. Most of the data is in <b>double</b> type or integer type, which is already good numerical representation.

The following are the existing string columns.
```
|-- tcp_flags: string (nullable = true)
|-- mqtt_conack_flags: string (nullable = true)
|-- mqtt_conflags: string (nullable = true)
|-- mqtt_hdrflags: string (nullable = true)
|-- mqtt_msg: string (nullable = true)
|-- mqtt_protoname: string (nullable = true)
|-- target: string (nullable = true)
```
### TODO: check out the meaning of the features

<!-- Here we changed the type of two flags since according to the paper and protocals, the flags will only take several fixed distince values, which are represented by flipping the ending bits of a integer size number. Therefore, it can always be represented in terms of a equivalent integer.

#### TODO: but this is not ordinal, this should be nominal -->

In [19]:
import pyspark.sql.functions as F

casted_types_df = renamed_columns_df
# casted_types_df = renamed_columns_df.withColumn(
#     "new_tcp_flags", 
#     F.expr("conv(substring(tcp_flags, 3), 16, 10)")
# ).drop("tcp_flags").distinct()      # deleting duplicate rows

# casted_types_df = renamed_columns_df.withColumn(
#     "new_mqtt_hdrflags", 
#     F.expr("conv(substring(mqtt_hdrflags, 3), 16, 10)")
# ).drop("mqtt_hdrflags").distinct()

# casted_types_df.show(2, vertical=True)
# casted_types_df.printSchema()

-RECORD 0--------------------------------
 tcp_flags                  | 0x00000018 
 tcp_time_delta             | 0.998867   
 tcp_len                    | 10         
 mqtt_conack_flags          | 0          
 mqtt_conack_flags_reserved | 0.0        
 mqtt_conack_flags_sp       | 0.0        
 mqtt_conack_val            | 0.0        
 mqtt_conflag_cleansess     | 0.0        
 mqtt_conflag_passwd        | 0.0        
 mqtt_conflag_qos           | 0.0        
 mqtt_conflag_reserved      | 0.0        
 mqtt_conflag_retain        | 0.0        
 mqtt_conflag_uname         | 0.0        
 mqtt_conflag_willflag      | 0.0        
 mqtt_conflags              | 0          
 mqtt_dupflag               | 0.0        
 mqtt_hdrflags              | 0x00000030 
 mqtt_kalive                | 0.0        
 mqtt_len                   | 8.0        
 mqtt_msg                   | 32         
 mqtt_msgid                 | 0.0        
 mqtt_msgtype               | 3.0        
 mqtt_proto_len             | 0.0 

Write the data into postgre table through pyspark jdbc API and verify the result with read command.

In [ ]:
db_properties={}
#update your db username
db_properties['username']="14813"
#update your db password
db_properties['password']="bigdata"
#make sure you got the right port number here
db_properties['url']= "jdbc:postgresql://localhost:5432/postgres"
#make sure you had the Postgres JAR file in the right location
db_properties['driver']="org.postgresql.Driver"
db_properties['table']= "mqtt"


df_train.write.format("jdbc")\
.mode("overwrite")\
.option("url", db_properties['url'])\
.option("dbtable", db_properties['table'])\
.option("user", db_properties['username'])\
.option("password", db_properties['password'])\
.option("Driver", db_properties['driver'])\
.save()

df_test.write.format("jdbc")\
.mode("append")\
.option("url", db_properties['url'])\
.option("dbtable", db_properties['table'])\
.option("user", db_properties['username'])\
.option("password", db_properties['password'])\
.option("Driver", db_properties['driver'])\
.save()


In [ ]:

sqlContext = SQLContext(spark.sparkContext)
df_read = sqlContext.read.format("jdbc")\
    .option("url", db_properties['url'])\
    .option("dbtable", db_properties['table'])\
    .option("user", db_properties['username'])\
    .option("password", db_properties['password'])\
    .option("Driver", db_properties['driver'])\
    .load()

df_read.show(1, vertical=True)